In [1]:
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Butterfly-Image-Classification-50-species'

In [2]:
def load_data():
    labels = {}
    labels_r = {}
    idx = -1
    data = []
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}/'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(112,112))
            data.append([img/255.0,idx])
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)
    X_train = torch.from_numpy(np.array(X_train)).float()
    y_train = torch.from_numpy(np.array(y_train)).float()
    X_test = torch.from_numpy(np.array(X_test)).float()
    y_test = torch.from_numpy(np.array(y_test)).float()
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [3]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|███████████████████████████████████████████| 50/50 [00:03<00:00, 14.05it/s]


In [4]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device)
y_test = torch.from_numpy(np.array(y_test)).to(device)

In [5]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [6]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [7]:
def get_pred(model):
    for file in os.listdir('./test_data/'):
        img = cv2.imread(f'./test_data/{file}')
        img = cv2.resize(img,(112,112))
        img = torch.from_numpy(np.array(img)).view(-1,3,112,112)
        preds = model(img.float())
        preds = torch.argmax(preds)
        preds = labels_r[preds]
        plt.figure(figsize=(12,6))
        plt.imshow(img)
        plt.title(f'{preds}')
        plt.savefig(f'./preds/{file}')
        plt.close()

In [8]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1d = Conv2d(3,6,(3,3))
        self.conv1dbatchnorm = BatchNorm2d(6)
        self.conv2d = Conv2d(6,9,(3,3))
        self.conv2dbatchnorm = BatchNorm2d(9)
        self.conv3d = Conv2d(9,12,(3,3))
        self.conv3dbatchnorm = BatchNorm2d(12)
        self.conv4d = Conv2d(12,15,(3,3))
        self.conv4dbatchnorm = BatchNorm2d(15)
        self.conv5d = Conv2d(15,18,(3,3))
        self.conv5dbatchnorm = BatchNorm2d(18)
        self.linear1 = Linear(18*1*1,128)
        self.linear1batchnorm = BatchNorm1d(128)
        self.linear2 = Linear(128,256)
        self.linear2batchnorm = BatchNorm1d(256)
        self.linear3 = Linear(256,idx+1)
    
    def forward(self,X):
        preds = self.activation(self.max_pool2d(self.conv1dbatchnorm(self.conv1d(X))))
        preds = self.activation(self.max_pool2d(self.conv2dbatchnorm(self.conv2d(preds))))
        preds = self.activation(self.max_pool2d(self.conv3dbatchnorm(self.conv3d(preds))))
        preds = self.activation(self.max_pool2d(self.conv4dbatchnorm(self.conv4d(preds))))
        preds = self.activation(self.max_pool2d(self.conv5dbatchnorm(self.conv5d(preds))))
#         print(preds.shape)
        preds = preds.view(-1,18*1*1)
        preds = self.activation(self.linear1batchnorm(self.linear1(preds)))
        preds = self.activation(self.linear2batchnorm(self.linear2(preds)))
        preds = self.linear3(preds)
        return preds

In [10]:
model = Model().to(device)

In [11]:
criterion = CrossEntropyLoss()

In [13]:
from torch.optim import *

In [14]:
optimizer = Adam(model.parameters(),lr=0.001)

In [15]:
epochs = 100

In [16]:
batch_size = 32

In [18]:
import wandb

In [19]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float().view(-1,3,112,112)
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    get_pred(model)
    for file in os.listdir('./preds/'):
        wandb.log({f'Img/{file}':wandb.Image(cv2.imread(f'./preds/{file}'))})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|                                                   | 0/100 [00:00<?, ?it/s]

torch.Size([32, 18, 1, 1])


RuntimeError: shape '[-1, 450]' is invalid for input of size 576